In [61]:
from pvlib.location import Location
from dotenv import load_dotenv
import googlemaps
from googlemaps.elevation import elevation
from googlemaps.timezone import timezone
import pandas as pd
import os

In [62]:
# initialize google client
load_dotenv()
google_apikey=os.getenv("google_api_key")
openWeather_apikey=os.getenv("open_weather_api")
solcast_apikey=os.getenv('solcast_apikey')
googlemap_client=googlemaps.Client(key=google_apikey)

In [63]:
# retrieve and process JSON data
# Location is in Decimal Degree coordinates
location1=Location(latitude=54.3395,longitude=-114.8016)
# result=elevation(client=googlemap_client,locations=(location1.latitude,location1.longitude))
result= timezone(client=googlemap_client,location=(location1.latitude,location1.longitude))
result['timeZoneId']


'America/Edmonton'

In [64]:
# in lib inverter files: https://github.com/NREL/SAM/tree/develop/deploy/libraries
# there are two ways of specifying data
from pvlib import pvsystem
modules = pvsystem.retrieve_sam('cecmod')
# customize parameters if module is not present in the database:
# https://pvlib-python.readthedocs.io/en/stable/user_guide/pvsystem.html#design-philosophy
# required weather options: 
# 1. Openweather API 
# 2. google solar API(geoTiff file)
# 3. Solcast


In [65]:
modules['AU_Optronics_PM060M00_275'].to_dict()

{'Technology': 'Mono-c-Si',
 'Bifacial': 0,
 'STC': 275.196,
 'PTC': 247.8,
 'A_c': 1.611,
 'Length': 1.639,
 'Width': 0.983,
 'N_s': 60,
 'I_sc_ref': 9.03,
 'V_oc_ref': 38.7,
 'I_mp_ref': 8.52,
 'V_mp_ref': 32.3,
 'alpha_sc': 0.005734,
 'beta_oc': -0.126781,
 'T_NOCT': 46.5,
 'a_ref': 1.659088,
 'I_L_ref': 9.032612,
 'I_o_ref': 6.641533e-10,
 'R_s': 0.170889,
 'R_sh_ref': 590.845032,
 'Adjust': 16.860441,
 'gamma_r': -0.4536,
 'BIPV': 'N',
 'Version': 'SAM 2018.11.11 r2',
 'Date': '1/3/2019'}

In [66]:
import requests
load_dotenv()

response=requests.get(f"https://pro.openweathermap.org/data/2.5/forecast/hourly",params={
    "lat":location1.latitude,
    "lon":location1.longitude,
    'appid':openWeather_apikey
})
response.json()

{'cod': '200',
 'message': 0,
 'cnt': 96,
 'list': [{'dt': 1727247600,
   'main': {'temp': 285.87,
    'feels_like': 285.29,
    'temp_min': 285.31,
    'temp_max': 285.87,
    'pressure': 1000,
    'sea_level': 1000,
    'grnd_level': 921,
    'humidity': 80,
    'temp_kf': 0.56},
   'weather': [{'id': 801,
     'main': 'Clouds',
     'description': 'few clouds',
     'icon': '02n'}],
   'clouds': {'all': 22},
   'wind': {'speed': 1.78, 'deg': 127, 'gust': 1.41},
   'visibility': 10000,
   'pop': 0,
   'sys': {'pod': 'n'},
   'dt_txt': '2024-09-25 07:00:00'},
  {'dt': 1727251200,
   'main': {'temp': 285.62,
    'feels_like': 285.06,
    'temp_min': 285.04,
    'temp_max': 285.62,
    'pressure': 999,
    'sea_level': 999,
    'grnd_level': 920,
    'humidity': 82,
    'temp_kf': 0.58},
   'weather': [{'id': 802,
     'main': 'Clouds',
     'description': 'scattered clouds',
     'icon': '03n'}],
   'clouds': {'all': 26},
   'wind': {'speed': 1.81, 'deg': 128, 'gust': 1.95},
   'visibi

In [67]:
import pvlib
# df, _, _, metadata = pvlib.iotools.get_pvgis_tmy(location1.latitude, location1.longitude, map_variables=True)


In [68]:
# pvlib.iotools.get_pvgis_hourly(location1.latitude,location1.longitude)

In [69]:
from datetime import datetime,timedelta
from zoneinfo import ZoneInfo

# Get the current time in UTC
now_utc = datetime.now(ZoneInfo('UTC'))
print("Current UTC time:", now_utc)

# Convert UTC time to another timezone (e.g., America/Edmonton)
now_edmonton = now_utc.astimezone(ZoneInfo('America/Edmonton'))
print("Current Edmonton time:", now_edmonton)
now=datetime.now(ZoneInfo('America/Edmonton'))
offset=now.utcoffset()
offset.total_seconds()/3600

now.isoformat()
yesterday=now-timedelta(days=1)
df, meta = pvlib.iotools.solcast.get_solcast_historic(
    latitude=location1.latitude,
    longitude=location1.longitude,
    start=yesterday.isoformat(),
    duration="P1D",
    api_key=solcast_apikey,
    time_zone=(offset.total_seconds()/3600),
    period="PT60M"
)
df

Current UTC time: 2024-09-25 06:06:17.779739+00:00
Current Edmonton time: 2024-09-25 00:06:17.779739-06:00


,temp_air,dni,ghi
period_mid,,,
2024-09-24 00:30:00-06:00,11,0,0
2024-09-24 01:30:00-06:00,11,0,0
2024-09-24 02:30:00-06:00,11,0,0
2024-09-24 03:30:00-06:00,11,0,0
2024-09-24 04:30:00-06:00,11,0,0
2024-09-24 05:30:00-06:00,10,0,0
2024-09-24 06:30:00-06:00,11,0,0
2024-09-24 07:30:00-06:00,11,41,8
2024-09-24 08:30:00-06:00,13,504,105


In [70]:
yesterday.timestamp()

1727157977.779739

In [71]:
df.index

DatetimeIndex(['2024-09-24 00:30:00-06:00', '2024-09-24 01:30:00-06:00',
               '2024-09-24 02:30:00-06:00', '2024-09-24 03:30:00-06:00',
               '2024-09-24 04:30:00-06:00', '2024-09-24 05:30:00-06:00',
               '2024-09-24 06:30:00-06:00', '2024-09-24 07:30:00-06:00',
               '2024-09-24 08:30:00-06:00', '2024-09-24 09:30:00-06:00',
               '2024-09-24 10:30:00-06:00', '2024-09-24 11:30:00-06:00',
               '2024-09-24 12:30:00-06:00', '2024-09-24 13:30:00-06:00',
               '2024-09-24 14:30:00-06:00', '2024-09-24 15:30:00-06:00',
               '2024-09-24 16:30:00-06:00', '2024-09-24 17:30:00-06:00',
               '2024-09-24 18:30:00-06:00', '2024-09-24 19:30:00-06:00',
               '2024-09-24 20:30:00-06:00', '2024-09-24 21:30:00-06:00',
               '2024-09-24 22:30:00-06:00', '2024-09-24 23:30:00-06:00',
               '2024-09-25 00:30:00-06:00'],
              dtype='datetime64[ns, UTC-06:00]', name='period_mid', freq=None)